In [1]:
import re
import os
import yaml
import json
import pandas as pd
from google.cloud import bigquery
import google.api_core.exceptions as google_exceptions

In [2]:
config  = yaml.safe_load(open('config.yaml', 'r'))

In [3]:
query_dir = 'queries'

In [4]:
queries = {}
for query_file in (os.listdir(query_dir)):
    with open(os.path.join(query_dir, query_file), 'r') as query:
        queries[query_file] = query.read()

In [5]:
query_client = bigquery.Client()

In [6]:
def exists_table(table_reference, client):
    try:
        client.get_table(table_reference)
        return True
    except google_exceptions.NotFound:
        return False

In [7]:
if exists_table(f'{config["project_id"]}.{config["dataset"]}.expanded_sub_properties', query_client):
    request = queries['all_new_buildings_that_have_not_been_expanded.sql']
else:
    request = queries['all_new_buildings.sql']

In [8]:
real_estate_raw = query_client.query(request).to_dataframe()

In [9]:
real_estate_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 5 columns):
ad_id             80 non-null int64
new_building      80 non-null bool
property_table    80 non-null object
property_attrs    80 non-null object
address           80 non-null object
dtypes: bool(1), int64(1), object(3)
memory usage: 2.7+ KB


In [10]:
real_estate_raw.set_index(keys='ad_id', drop=True, inplace=True)

In [11]:
real_estate_raw.head()

,new_building,property_table,property_attrs,address
ad_id,,,,
84585697,True,"{""Bolig"": [""05""], ""Soverom"": [""3""], ""Etg"": [""2...","{""Omkostninger"": ""10\u00a0002 kr"", ""Totalpris""...","Blomsterbakken 28, 1487 Hakadal"
86180027,True,"{""Bolig"": [""101"", ""113"", ""201"", ""205"", ""206"", ...","{""Totalpris"": ""3\u00a0090\u00a0000 \u2013 4\u0...","Grensevegen 13, 1929 Auli"
87708616,True,"{""Bolig"": [""37"", ""46"", ""47"", ""52"", ""54""], ""Sov...","{""Omkostninger"": ""12\u00a0222 kr"", ""Totalpris""...","Furumo - Furumogrenda, 1400 Ski"
88980906,True,"{""Bolig"": [""D1 08"", ""D1 12"", ""D1 17"", ""D1 19"",...","{""Omkostninger"": ""19\u00a0442 \u2013 24\u00a06...","Krydderhagen - Hus D1-D2, 0579 Oslo"
88990917,True,"{""Bolig"": [""1306"", ""1403"", ""1506"", ""3502"", ""35...","{""Fellesgjeld"": ""659\u00a0000 \u2013 779\u00a0...","Ensjøhøyden, Stålverksveien 1, 0661 Oslo"


In [12]:
for col in ['property_table', 'property_attrs']:
    real_estate_raw[col] = [json.loads(cell) for cell in real_estate_raw[col]]

In [13]:
def expanand_property_tables(new_buildings):
    cols = new_buildings.iloc[0]['property_table'].keys()
    col_map = {'Bolig': 'apt_id',
               'Soverom': 'num_bedrooms',
               'Etg': 'floor',
               'P-rom': 'primary_size',
               'BRA': 'total_size',
               'Totalpris': 'price',
               'Pris': 'price'}
    property_tables = pd.DataFrame(columns=cols)
    for code, row in new_buildings['property_table'].iteritems():
        all_properties = [arr for arr in row.values()]
        to_append = dict(zip(cols, all_properties))
        frame = pd.DataFrame(to_append, columns=cols)
        frame['ad_id'] = code
        property_tables = property_tables.append(frame, ignore_index=True, sort=False)
    property_tables.rename(columns=col_map, inplace=True)
    return property_tables

In [14]:
sub_properties = expanand_property_tables(real_estate_raw[(real_estate_raw.new_building == True) & (real_estate_raw.property_table)])

In [15]:
sub_properties.head()

,apt_id,num_bedrooms,floor,primary_size,total_size,price,ad_id
0,05,3,2.,84 m²,89 m²,4 108 002 kr,84585697.0
1,101,2,1.,74 m²,77 m²,3 190 000 kr,86180027.0
2,113,2,1.,80 m²,82 m²,3 090 000 kr,86180027.0
3,201,3,2.,97 m²,100 m²,4 250 000 kr,86180027.0
4,205,2,2.,74 m²,77 m²,3 290 000 kr,86180027.0


In [16]:
def get_int_from_str(string):
    try:
        concat_string = ''.join(re.findall(r'[\d.]', string))
        if concat_string != '':
            return float(concat_string)
    except Exception as e:
        print(e, string)
        pass

In [17]:
sub_properties_parsed = sub_properties.copy()
for col in ['primary_size', 'floor', 'price', 'num_bedrooms', 'total_size']:
    sub_properties_parsed[col] = [get_int_from_str(s) for s in sub_properties_parsed[col]]

expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None


In [18]:
sub_properties_parsed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 935 entries, 0 to 934
Data columns (total 7 columns):
apt_id          935 non-null object
num_bedrooms    930 non-null float64
floor           850 non-null float64
primary_size    807 non-null float64
total_size      934 non-null float64
price           930 non-null float64
ad_id           935 non-null float64
dtypes: float64(6), object(1)
memory usage: 51.2+ KB


In [19]:
sub_properties_parsed.head()

,apt_id,num_bedrooms,floor,primary_size,total_size,price,ad_id
0,05,3.0,2.0,84.0,89.0,4108002.0,84585697.0
1,101,2.0,1.0,74.0,77.0,3190000.0,86180027.0
2,113,2.0,1.0,80.0,82.0,3090000.0,86180027.0
3,201,3.0,2.0,97.0,100.0,4250000.0,86180027.0
4,205,2.0,2.0,74.0,77.0,3290000.0,86180027.0


In [20]:
sub_properties_parsed.drop_duplicates(inplace=True)

In [21]:
sub_properties_parsed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 920 entries, 0 to 934
Data columns (total 7 columns):
apt_id          920 non-null object
num_bedrooms    915 non-null float64
floor           835 non-null float64
primary_size    807 non-null float64
total_size      919 non-null float64
price           915 non-null float64
ad_id           920 non-null float64
dtypes: float64(6), object(1)
memory usage: 57.5+ KB


In [22]:
sub_properties_parsed.to_gbq(destination_table='housing_data.expanded_sub_properties',
                             project_id='hde-test-clean',
                             if_exists='append')